In [ ]:
import pandas as pd
import json
import datetime


In [ ]:
class Config():
    def __init__(self):
      # self.hattrick_candles = 1 # 1min
      # self.hatrick_freq = '1min'
      # self.double_candles = 15 # 15min
      # self.double_freq = '15min'
      # self.trigger_candles = 240 # 4h
      # self.trigger_freq = '4h'
      # self.structure_candles = 10080 # 1W
      # self.structure_freq = '1W'

      self.time_shifter = {
          'structure': 0,
          'pattern': -1,
          'trigger': -2,
          'double': -4,
          'hattrick': -6,
          }

      self.times = [(1, '1min'), (5, '5min'), (15, '15min'), (60, '1H'), (240, '4H'), (60*24, '1D'), (60*24*7, '1W')]
      self.hattrick_index = 0
      self.trigger_dept = 16
      
      self.dept_of_analysis = 3
      self.feature_columns = ['open', 'high', 'low', 'close', 'volume']

      self.end_time = '2021-03-01 03:43:00'

      self.id = datetime.datetime.now().strftime("%Y-%m-%d.%H-%M")


class MyEncoder(json.JSONEncoder):
    def default(self, o):
        return o.__dict__


myEncoder = MyEncoder()
debug = False

config = Config()
print(myEncoder.encode(config))

colab_ip = %system hostname -I   # uses colab magic to get list from bash
colab_ip = colab_ip[0].strip()   # returns "172.28.0.12"
colab_port = 9000                # could use 6000, 8080, or 9000

import requests
filename = requests.get(f"http://{colab_ip}:{colab_port}/api/sessions").json()[0]["name"]
# Output will be of the form "xyz.ipynb"
print(filename)

with open(f'{filename}.{config.id}.config.txt', 'w') as config_file:
    config_file.write(myEncoder.encode(config))

{"time_shifter": {"structure": 0, "pattern": -1, "trigger": -2, "double": -4, "hattrick": -6}, "times": [[1, "1min"], [5, "5min"], [15, "15min"], [60, "1H"], [240, "4H"], [1440, "1D"], [10080, "1W"]], "hattrick_index": 0, "trigger_dept": 16, "dept_of_analysis": 3, "feature_columns": ["open", "high", "low", "close", "volume"], "end_time": "2021-03-01 03:43:00", "id": "2023-06-07.06-46"}


TREX.Moghadam.DataPreparation.1.ipynb


In [ ]:
# ! wget https: // raw.githubusercontent.com/b-raisdana/BTC-price/37667a09695d26f8cf303c4989d0dd0a51ab18d0/A.BTC-2021min.01-01.00-01-00.to.03-01.03-43-00.csv.zip -O A.BTC-2021min.01-01.00-01-00.to.03-01.03-43-00.csv.zip
! wget https://raw.githubusercontent.com/b-raisdana/BTC-price/main/17-01-01.0-01TO17-12-31.23-59.zip -O 17-01-01.0-01TO17-12-31.23-59.zip
! unzip -o 17-01-01.0-01TO17-12-31.23-59.zip

--2023-06-07 06:46:23--  https://raw.githubusercontent.com/b-raisdana/BTC-price/main/17-01-01.0-01TO17-12-31.23-59.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8114787 (7.7M) [application/zip]
Saving to: ‘17-01-01.0-01TO17-12-31.23-59.zip’

17-01-01.0-01TO17-1 100%[===================>]   7.74M  --.-KB/s    in 0.1s    

2023-06-07 06:46:24 (75.2 MB/s) - ‘17-01-01.0-01TO17-12-31.23-59.zip’ saved [8114787/8114787]

Archive:  17-01-01.0-01TO17-12-31.23-59.zip
  inflating: 17-01-01.0-01TO17-12-31.23-59.csv  


In [ ]:
import plotly.graph_objects as plgo
DEBUG = False
def plotfig(data: pd, save: bool = True, name: str = ''):
  import os 
  MAX_LEN_OF_DATA_FRAME_TO_PLOT = 50000
  SAFE_LEN_OF_DATA_FRAME_TO_PLOT = 10000
  if len(data.index) > MAX_LEN_OF_DATA_FRAME_TO_PLOT:
      raise Exception(f'Too many rows to plt ({len(data.index),}>{MAX_LEN_OF_DATA_FRAME_TO_PLOT})') 
  if len(data.index) > SAFE_LEN_OF_DATA_FRAME_TO_PLOT:
      print(f'Plotting too much data will slow us down ({len(data.index),}>{SAFE_LEN_OF_DATA_FRAME_TO_PLOT})') 
  if not os.path.isfile('kaleido.installed'):
      print('kaleido not satisfied!')
      try:
          ! pip install -q condacolab
          import condacolab

          if not condacolab.check():
              condacolab.install()
              ! conda install -c conda-forge python-kaleido
              ! echo '' > kaleido.installed
          else:
              print('condacolab already satisfied')
      except:
          ! pip install -U kaleido
          ! echo '' > kaleido.installed
  if DEBUG: print(f'data({data.shape})')
  if DEBUG: print(data)
  fig = plgo.Figure(data=[plgo.Candlestick(x=data.index.values,
                                         open=data['open'], high=data['high'], low=data['low'],
                                         close=data['close'], 
                                         )], ).update_yaxes(fixedrange=False).update_layout(yaxis_title=name)
  if DEBUG: print('1')
  fig.show() 
  if DEBUG: print('2')
  if save: fig.write_image(f'{config.id}.{name}.png')

In [ ]:
reverse_prices = pd.read_csv('17-01-01.0-01TO17-12-31.23-59.csv', index_col='date', parse_dates=['date'])
reverse_prices = reverse_prices.rename(columns={'Volume BTC': 'volume'})
reverse_prices = reverse_prices[config.feature_columns]
print(reverse_prices)
plotfig(reverse_prices.head(10000), name='reverse_prices Head')
plotfig(reverse_prices.tail(10000), name='reverse_prices Tail')

                         open      high       low     close    volume
date                                                                 
2017-12-31 23:59:00  13913.28  13913.28  13867.18  13880.00  0.591748
2017-12-31 23:58:00  13913.26  13953.83  13884.69  13953.77  1.398784
2017-12-31 23:57:00  13908.73  13913.26  13874.99  13913.26  0.775012
2017-12-31 23:56:00  13827.00  13908.69  13827.00  13859.58  0.666459
2017-12-31 23:55:00  13825.05  13825.05  13825.05  13825.05  0.065501
...                       ...       ...       ...       ...       ...
2017-01-01 00:05:00    966.43    966.43    966.43    966.43  0.107000
2017-01-01 00:04:00    966.37    966.37    966.37    966.37  0.000000
2017-01-01 00:03:00    966.37    966.37    966.37    966.37  0.000000
2017-01-01 00:02:00    966.16    966.37    966.16    966.37  8.087376
2017-01-01 00:01:00    966.34    966.34    966.34    966.34  7.610000

[525599 rows x 5 columns]


In [ ]:
for bar_size, period_name in config.times:
    summarized_prices = reverse_prices.groupby(pd.Grouper(freq=period_name))\
        .agg({'open': 'first', 
                'close': 'last', 
                'low': 'min', 
                'high': 'max',
                'volume':'sum'})
    plotfig(summarized_prices.head(10000),name=period_name)
    summarized_prices.to_csv(f'17-01-01.0-01TO17-12-31.23-59.{period_name}.zip', compression='zip')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# hattrick_reverse_prices = reverse_prices.head(config.hattrick_candles*pow(16, config.dept_of_analysis))
# plotfig(hattrick_reverse_prices, name='hattrick_reverse_prices')
# double_reverse_prices = reverse_prices.head(config.double_candles*pow(16, config.dept_of_analysis)).groupby(pd.Grouper(freq=config.double_freq))\
#       .agg({'open': 'first', 
#             'close': 'last', 
#             'low': 'min', 
#             'high': 'max',
#             'volume':'sum'})
# plotfig(double_reverse_prices,name='double_reverse_prices')
# trigger_reverse_prices = reverse_prices.head(config.trigger_candles*pow(16, config.dept_of_analysis)).groupby(pd.Grouper(freq=config.trigger_freq))\
#       .agg({'open': 'first', 
#             'close': 'last',
#             'low': 'min', 
#             'high': 'max',
#             'volume':'sum'})
# plotfig(trigger_reverse_prices,name='trigger_reverse_prices')
# structure_reverse_prices = reverse_prices.head(config.structure_candles*pow(16, config.dept_of_analysis)).groupby(pd.Grouper(freq=config.structure_freq))\
#       .agg({'open': 'first', 
#             'close': 'last', 
#             'low': 'min', 
#             'high': 'max',
#             'volume':'sum'})
# plotfig(structure_reverse_prices,name='structure_reverse_prices')
